# Cryptocurrencies explorations

## Introduction

The main goal of this computational Markdown document is to provide some basic views and insights into the landscape of cryptocurrencies. 
The “landscape” we consider consists of price action and trading volume time series for cryptocurrencies found in Yahoo Finance.

In this document we compute and plot with Raku the statistics in [AA1].

### Details on "running" the document

The Raku package used for data retrieval is "Data::Cryptocurrencies", [AAp1].
The JavaScript D3 plots are made via "JavaScript::D3", [AAp2].

This Markdown document is converted to its woven Markdown version with "Text::CodeProcessing", [AAp3].
The woven Markdown document is converted to an HTML document with "Markdown::Grammar", [AAp4].

Here is the corresponding shell command:

```
file-code-chunks-eval Cryptocurrencies-explorations.md && 
  from-markdown Cryptocurrencies-explorations_woven.md -t html -o Cryptocurrencies-explorations.html && 
  open Cryptocurrencies-explorations.html
```

-------

## Setup

Here we load the packages used below:

In [1]:
use Data::Cryptocurrencies;
use Data::Reshapers;
use Data::Summarizers;
use Text::Plot;
use JavaScript::D3;

Javascript D3 plots setup:

In [2]:
%% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

-------

## Time series

Here we get Bitcoin (BTC), Ethereum (ETH), and Bitcoin Cash (BCH) data from 1/1/2020 until now:

In [3]:
my %ccTS = cryptocurrency-data(<BTC ETH BCH>, dates => (DateTime.new(2020,1,1,0,0,0), now), props => <DateTime Close>, format => 'hash'):cache-all;

say %ccTS>>.elems;

{BCH => 1450, BTC => 1450, ETH => 1450}


Clean data:

In [4]:
%ccTS = %ccTS.map({ $_.key => $_.value.grep({ $_<Close> ~~ Numeric }).List });
%ccTS>>.elems

{BCH => 1449, BTC => 1449, ETH => 1449}

Here is a summary:

In [5]:
records-summary(%ccTS);
%ccTS>>.elems

summary of ETH =>
+---------------------------------------+----------------------------+
| DateTime                              | Close                      |
+---------------------------------------+----------------------------+
| Min    => 2020-01-01T00:00:37Z        | Min    => 110.605873       |
| 1st-Qu => 2020-12-27T12:00:37Z        | 1st-Qu => 706.504944       |
| Mean   => 2021-12-25T00:01:36.627329Z | Mean   => 1711.41879177157 |
| Median => 2021-12-25T00:00:37Z        | Median => 1667.943848      |
| 3rd-Qu => 2022-12-22T12:00:37Z        | 3rd-Qu => 2244.32312       |
| Max    => 2023-12-20T00:00:37Z        | Max    => 4812.087402      |
+---------------------------------------+----------------------------+
summary of BCH =>
+---------------------------------------+------------------------+
| DateTime                              | Close                  |
+---------------------------------------+------------------------+
| Min    => 2020-01-01T00:00:37Z        | Min    => 8

{BCH => 1449, BTC => 1449, ETH => 1449}

Here we prepare data for date-list D3.js plots:

In [6]:
my %tsAll = %ccTS.map({ $_.key => $_.value.map(-> %r { %( date => %r<DateTime>.Str.substr(0,10), value => %r<Close>.Numeric, group => $_.key) }).grep({ $_.<value> ~~ Numeric }) });
say %tsAll>>.elems;
my @dsAll = reduce(-> @a, @v { @a.append(@v) }, [[], |%tsAll.values]);
say @dsAll.elems;

{BCH => 1449, BTC => 1449, ETH => 1449}
4347


Here we plot all time series:

In [7]:
%% js
js-d3-date-list-plot(@dsAll, plot-label => 'Cryptocurrencies', width => 800, height => 300, format => 'jupyter', div-id => 'BTC', grid-lines=>(10,5));

The "all together" plot is not that informative, so we plot the time series separately:

In [8]:
%% js
js-d3-date-list-plot(%tsAll<BTC>.map(*<date value>), plot-label => 'BTC', width => 800, height => 300, format => 'jupyter', div-id => 'BTC', grid-lines=>(10,5));

In [9]:
%% js 
js-d3-date-list-plot(%tsAll<ETH>.map(*<date value>), plot-label => 'ETH', width => 800, height => 300, format => 'jupyter', div-id => 'ETH', grid-lines=>(10,5));

In [10]:
%% js 
js-d3-date-list-plot(%tsAll<BCH>.map(*<date value>), plot-label => 'BCH', width => 800, height => 300, format => 'jupyter', div-id => 'ETH', grid-lines=>(10,5));

-------

## Pareto principle adherence

Get data for all cryptocurrencies for the last 90 days:

In [11]:
my @dsData = cryptocurrency-data('all', dates => (now - 90 * 24 * 3600, now), props => ['Symbol', 'DateTime', 'Close', 'Volume'], format => 'dataset');
say "Dimensions : {dimensions(@dsData)}.";

Dimensions : 1979 4.


Clean data and show summary:

In [12]:
@dsData = @dsData.grep({ $_<Close> ~~ Numeric });
records-summary(@dsData);
dimensions(@dsData)

+---------------------------+---------------------------------------+-----------------------------+-----------------+
| Close                     | DateTime                              | Volume                      | Symbol          |
+---------------------------+---------------------------------------+-----------------------------+-----------------+
| Min    => 0.002935        | Min    => 2023-09-22T00:00:37Z        | Min    => 228152            | USDT    => 89   |
| 1st-Qu => 0.257607        | 1st-Qu => 2023-10-14T00:00:37Z        | 1st-Qu => 89386694.5        | DOGE    => 89   |
| Mean   => 1762.9311261204 | Mean   => 2023-11-05T00:16:47.786517Z | Mean   => 3357500310.561263 | ETC     => 89   |
| Median => 1.000028        | Median => 2023-11-05T00:00:37Z        | Median => 234557830         | BNB     => 89   |
| 3rd-Qu => 71.983173       | 3rd-Qu => 2023-11-27T00:00:37Z        | 3rd-Qu => 874998909         | XRP     => 89   |
| Max    => 44166.601563    | Max    => 2023-12-20T00:00

(1869 4)

Group by "Symbol" and find price- and volume totals per group:

In [13]:
my %groups = group-by(@dsData, "Symbol");

my %prices = %groups.map({ $_.key => $_.value.map(*<Close>).sum });
my %volumes = %groups.map({ $_.key => $_.value.map(*<Volume>).sum });

say %volumes.sort({ -$_.value }).head(5);

(USDT => 3086852864832 BTC => 1596741037649 ETH => 763883585934 USDC => 364605361534 XRP => 113548549382)


Here is the Pareto plot for closing prices:

In [14]:
%%js 
my @ps = pareto-principle-statistic(%prices);
my @pBubbleSpec = @ps.map({ %(x => $k++, y => $_.value, z => 0, group => $_.key) });
js-d3-bubble-chart(@pBubbleSpec, plot-label => 'Pareto principle adherence for closing prices'):grid-lines;

Here is the Pareto plot for trading volumes:

In [15]:
%%js
my @ps = pareto-principle-statistic(%volumes);
my @pBubbleSpec = @ps.map({ %(x => $k++, y => $_.value / max(@ps>>.value), z => 0, group => $_.key) });
js-d3-bubble-chart(@pBubbleSpec, plot-label => 'Pareto principle adherence for trading volumes'):grid-lines;

-------

## References

### Articles

[AA1] Anton Antonov
["Crypto-currencies data acquisition with visualization"](https://mathematicaforprediction.wordpress.com/2021/06/19/crypto-currencies-data-acquisition-with-visualization/),
(2021),
[MathematicaForPrediction at WordPress](https://mathematicaforprediction.wordpress.com).

[AA2] Anton Antonov
["Cryptocurrencies data explorations"](https://mathematicaforprediction.wordpress.com/2021/06/22/cryptocurrencies-data-explorations/),
(2021),
[MathematicaForPrediction at WordPress](https://mathematicaforprediction.wordpress.com).

### Functions, packages

[AAf1] Anton Antonov,
[CryptocurrencyData](https://www.wolframcloud.com/obj/antononcube/DeployedResources/Function/CryptocurrencyData/) Mathematica resource function,
(2021).
[WolframCloud/antononcube](https://www.wolframcloud.com/obj/antononcube).

[AAp1] Anton Antonov,
[Data::Cryptocurrencies Raku package](https://github.com/antononcube/Raku-Data-Cryptocurrencies),
(2023).
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov,
[JavaScript::D3 Raku package](https://github.com/antononcube/Raku-JavaScript-D3),
(2022).
[GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonov,
[Text::CodeProcessing Raku package](https://github.com/antononcube/Raku-Text-CodeProcessing),
(2021).
[GitHub/antononcube](https://github.com/antononcube).

[AAp4] Anton Antonov,
[Markdown::Grammar](https://github.com/antononcube/Raku-Markdown-Grammar),
(2022).
[GitHub/antononcube](https://github.com/antononcube).
